In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load model

alexnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
params = {name: parameter for name, parameter in alexnet.named_parameters()}

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [5]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000, hidden_keys=[]):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 3, stride = 2, dilation=1, ceil_mode=False),

            nn.Conv2d(64, 192, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 3, stride = 2, dilation=1, ceil_mode=False),

            nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 3, stride = 2, dilation=1, ceil_mode=False),

            nn.Flatten(),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5, inplace=False),
            nn.Linear(9216, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(0.5, inplace=False),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            nn.Linear(4096, num_classes)
        )

        # initialize hidden info
        self.hidden_info = {}
        self.update_hidden_keys(hidden_keys)
        
    def forward(self, x):
        x = x.float()
        for i, child in enumerate(list(self.features.children())):
            x = child(x)
            if i in self.hidden_keys: self.hidden_info[i].append(x)
        out = self.classifier(x)
        return out

    def update_hidden_keys(self, keys):
        dic = {}
        self.hidden_keys = keys
        for key in self.hidden_keys: dic[key] = []
        self.hidden_info = {**dic, **self.hidden_info} # update, with priority given to existing self.hidden_info

In [10]:
model = AlexNet(hidden_keys=[0, 1, 2])
model.load_state_dict(params)

<All keys matched successfully>

In [9]:
print(model) 
# Conv layers: 0, 3, 6, 8, 10
# "End" layers: 2, 5, 7, 9, 12

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): 

In [11]:
X = np.random.normal(0, 1, size=(2, 3, 224, 224))
X = torch.from_numpy(X).float()

out = model(X)

In [15]:
print(
    model.hidden_info[0][0].shape,
    model.hidden_info[1][0].shape,
    model.hidden_info[2][0].shape,
)

torch.Size([2, 64, 55, 55]) torch.Size([2, 64, 55, 55]) torch.Size([2, 64, 27, 27])
